In [1]:
%matplotlib inline
import random
import numpy as np
from d2l import torch as d2l

# 17.1 Markov Decision Process (MDP)

Markov decision process (MDP), 
$$\textrm{MDP}: (\mathcal{S}, \mathcal{A}, T, r)$$
-  $S$ be the set of states in the MDP
- $A$ be the set of actions that the robot can take at each state. Actions can change the current state of the robot to some other state within the set $S$
- Transition Function, $T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \to [0,1]$ such that $T(s, a, s') = P(s' \mid s, a)$ using the conditional probability of reaching a state $s'$ given that the robot was at state $s$ and took an action $a$
    - $\sum_{s' \in \mathcal{S}} T(s, a, s') = 1$ for all $s \in \mathcal{S}$ and $a \in \mathcal{A}$, i.e., the robot has to go to some state if it takes an action
-  Reward, $r: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$. We say that the robot gets a reward $r(s,a)$ if the robot takes an action $a$ at state $s$ 
- Trajectory, $\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots)$
    - At each time step $t$ the robot is at a state $s_t$ and takes an action $a_t$ which results in a reward $r_t = r(s_t, a_t)$. The discounted return of a trajectory is the total reward obtained by the robot along such a trajectory
    $$R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t$$
    - goal of RL is to find a trajectory that has the largest return

# 17.2 Value Iteration

- Stochastic Policy denoted as $\pi(a \mid s)$ (policy for short) is a conditional distribution over the actions $a \in \mathcal{A}$ given the state $s \in \mathcal{S}$, $\pi(a \mid s) \equiv P(a \mid s)$
    - $\sum_a \pi(a \mid s) = 1$ for any state $s$
    - optimal policy, $\pi^* = \underset{\pi}{\mathrm{argmax}} V^\pi(s_0).$
- Value Function - the expected $\gamma$-discounted return obtained by the robot if it begins at state $s_0$ and takes actions from the policy $\pi$ at each time instant.
$$V^\pi(s_0) = E_{a_t \sim \pi(s_t)} \Big[ R(\tau) \Big] = E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \Big] = r(s_0, a_0) + \gamma\ E_{a_0 \sim \pi(s_0)} \Big[ E_{s_1 \sim P(s_1 \mid s_0, a_0)} \Big[ V^\pi(s_1) \Big] \Big].$$
$$V^\pi(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi(s') \Big];\ \textrm{for all } s \in \mathcal{S}.$$
- Action-Value Function - the average return of a trajectory that begins at $s_0$ but when the action of the first stage is fixed to be $a_0$
$$Q^\pi(s_0, a_0) = r(s_0, a_0) + E_{a_t \sim \pi(s_t)} \Big[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \Big],$$
$$Q^\pi(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s' \mid s, a) \sum_{a' \in \mathcal{A}} \pi(a' \mid s')\ Q^\pi(s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.$$

## 17.6&7 Value Iteration & Policy Evalutation
- value iteration$$V_{k+1}(s) = \max_{a \in \mathcal{A}} \Big\{ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V_k(s') \Big\};\ \textrm{for all } s \in \mathcal{S}.$$
    - as $k \to \infty$ the value function estimated by the Value Iteration algorithm converges to the optimal value function irrespective of the initialization $V_0$
    $$V^*(s) = \lim_{k \to \infty} V_k(s);\ \textrm{for all states } s \in \mathcal{S}.$$
- action-value function iteration 
$$Q_{k+1}(s, a) = r(s, a) + \gamma \max_{a' \in \mathcal{A}} \sum_{s' \in \mathcal{S}} P(s' \mid s, a) Q_k (s', a');\ \textrm{ for all } s \in \mathcal{S}, a \in \mathcal{A}.$$
$$Q^*(s, a) = \lim_{k \to \infty} Q_k(s, a)$$
- Policy Evaluation
$$V^\pi_{k+1}(s) = \sum_{a \in \mathcal{A}} \pi(a \mid s) \Big[ r(s,  a) + \gamma\  \sum_{s' \in \mathcal{S}} P(s' \mid s, a) V^\pi_k(s') \Big];\ \textrm{for all } s \in \mathcal{S}.$$
$$V^\pi(s) = \lim_{k \to \infty} V^\pi_k(s);\ \textrm{for all states } s \in \mathcal{S}.$$
- !!!! implementing the Value Iteration algorithm requires that we know the Markov decision process (MDP), e.g., the transition and reward functions, completely.


In [ ]:
seed = 0  # Random number generator seed
gamma = 0.95  # Discount factor
num_iters = 10  # Number of iterations
random.seed(seed)  # Set the random seed to ensure results can be reproduced
np.random.seed(seed)

# Now set up the environment
env_info = d2l.make_env("FrozenLake-v1", seed=seed)

In [ ]:
def value_iteration(env_info, gamma, num_iters):
    env_desc = env_info["desc"]  # 2D array shows what each item means
    prob_idx = env_info["trans_prob_idx"]
    nextstate_idx = env_info["nextstate_idx"]
    reward_idx = env_info["reward_idx"]
    num_states = env_info["num_states"]
    num_actions = env_info["num_actions"]
    mdp = env_info["mdp"]

    V = np.zeros((num_iters + 1, num_states))
    Q = np.zeros((num_iters + 1, num_states, num_actions))
    pi = np.zeros((num_iters + 1, num_states))

    for k in range(1, num_iters + 1):
        for s in range(num_states):
            for a in range(num_actions):
                # Calculate \sum_{s'} p(s'\mid s,a) [r + \gamma v_k(s')]
                for pxrds in mdp[(s, a)]:
                    # mdp(s,a): [(p1,next1,r1,d1),(p2,next2,r2,d2),..]
                    pr = pxrds[prob_idx]  # p(s'\mid s,a)
                    nextstate = pxrds[nextstate_idx]  # Next state
                    reward = pxrds[reward_idx]  # Reward
                    Q[k, s, a] += pr * (reward + gamma * V[k - 1, nextstate])
            # Record max value and max action
            V[k, s] = np.max(Q[k, s, :])
            pi[k, s] = np.argmax(Q[k, s, :])
    d2l.show_value_function_progress(env_desc, V[:-1], pi[:-1])


value_iteration(env_info=env_info, gamma=gamma, num_iters=num_iters)

# 17.3 Q-Learning (类似MC simulation和greedy algo)

- Q-Learning is an algorithm to learn the value function without necessarily knowing the MDP
- approximate version of Q function
$$\hat{Q} = \min_Q \underbrace{\frac{1}{nT} \sum_{i=1}^n \sum_{t=0}^{T-1} (Q(s_t^i, a_t^i) - r(s_t^i, a_t^i) - \gamma \max_{a'} Q(s_{t+1}^i, a'))^2}_{\stackrel{\textrm{def}}{=} \ell(Q)}.$$
- update Q function $$\begin{split}\begin{aligned}Q(s_t^i, a_t^i) &\leftarrow Q(s_t^i, a_t^i) - \alpha \nabla_{Q(s_t^i,a_t^i)} \ell(Q) 
\\&=(1 - \alpha) Q(s_t^i,a_t^i) - \alpha \Big( r(s_t^i, a_t^i) + \gamma (1 - \mathbb{1}_{s_{t+1}^i \textrm{ is terminal}} )\max_{a'} Q(s_{t+1}^i, a') \Big).,\end{aligned}\end{split}$$
    -  The value of state-action tuples $(s, a)$ that are not a part of the dataset is set to $-\infty$
-  $\epsilon$-greedy exploration policy
$$\begin{split}\pi_e(a \mid s) = \begin{cases}\mathrm{argmax}_{a'} \hat{Q}(s, a') & \textrm{with prob. } 1-\epsilon \\ \textrm{uniform}(\mathcal{A}) & \textrm{with prob. } \epsilon,\end{cases}\end{split}$$
- softmax exploration policy
$$\pi_e(a \mid s) = \frac{e^{\hat{Q}(s, a)/T}}{\sum_{a'} e^{\hat{Q}(s, a')/T}};$$

In [ ]:
seed = 0  # Random number generator seed
gamma = 0.95  # Discount factor
num_iters = 256  # Number of iterations
alpha = 0.9  # Learing rate
epsilon = 0.9  # Epsilon in epsilion gready algorithm
random.seed(seed)  # Set the random seed
np.random.seed(seed)

# Now set up the environment
env_info = d2l.make_env("FrozenLake-v1", seed=seed)

In [ ]:
def e_greedy(env, Q, s, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()

    else:
        return np.argmax(Q[s, :])

In [ ]:
def q_learning(env_info, gamma, num_iters, alpha, epsilon):
    env_desc = env_info["desc"]  # 2D array specifying what each grid item means
    env = env_info["env"]  # 2D array specifying what each grid item means
    num_states = env_info["num_states"]
    num_actions = env_info["num_actions"]

    Q = np.zeros((num_states, num_actions))
    V = np.zeros((num_iters + 1, num_states))
    pi = np.zeros((num_iters + 1, num_states))

    for k in range(1, num_iters + 1):
        # Reset environment
        state, done = env.reset(), False
        while not done:
            # Select an action for a given state and acts in env based on selected action
            action = e_greedy(env, Q, state, epsilon)
            next_state, reward, done, _ = env.step(action)

            # Q-update:
            y = reward + gamma * np.max(Q[next_state, :])
            Q[state, action] = Q[state, action] + alpha * (y - Q[state, action])

            # Move to the next state
            state = next_state
        # Record max value and max action for visualization purpose only
        for s in range(num_states):
            V[k, s] = np.max(Q[s, :])
            pi[k, s] = np.argmax(Q[s, :])
    d2l.show_Q_function_progress(env_desc, V[:-1], pi[:-1])


q_learning(
    env_info=env_info, gamma=gamma, num_iters=num_iters, alpha=alpha, epsilon=epsilon
)